 Task 2.1 Create StreamingContext


In [ ]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

In [ ]:
sc = SparkContext("local[2]", "StreamingExample")

In [ ]:
ssc = StreamingContext(sc, 10)

In [ ]:
print("StreamingContext created with 2 threads and 10s batch interval.")

Task 2.2: TCP Source via Netcat

In [ ]:
# Create DStream that connects to localhost:9999
lines = ssc.socketTextStream("localhost", 9999)

# Simple transformation: split lines into words
words = lines.flatMap(lambda line: line.split())

# Count each word
pairs = words.map(lambda word: (word, 1))
word_counts = pairs.reduceByKey(lambda x, y: x + y)

# Print result
word_counts.pprint()

# Start the streaming context and wait for termination
ssc.start()
ssc.awaitTermination()

Task 2.3: Generate k-mers

In [ ]:
def generate_kmers(text, k=3):
    return [text[i:i+k] for i in range(len(text) - k + 1)]

# Split lines into words, then generate k-mers from each word
kmers = lines.flatMap(lambda line: line.split()) \
             .flatMap(lambda word: generate_kmers(word, 3))

# Count k-mer frequency
kmer_counts = kmers.map(lambda kmer: (kmer, 1)) \
                   .reduceByKey(lambda a, b: a + b)

# Display the k-mer counts
kmer_counts.pprint()

# Start and await termination
ssc.start()
ssc.awaitTermination()

Task 2.4: Count k-mers

In [ ]:
def generate_kmers(word, k=3):
    return [word[i:i+k] for i in range(len(word) - k + 1)]

# Generate k-mers from each line, then count
kmers = lines.flatMap(lambda line: line.split()) \
             .flatMap(lambda word: generate_kmers(word, 3)) \
             .map(lambda kmer: (kmer, 1)) \
             .reduceByKey(lambda a, b: a + b)

# Print result
kmers.pprint()

# Start streaming
ssc.start()
ssc.awaitTermination()